In [1]:
import collections
import dataclasses
import itertools
import json
import numpy as np
import os
import pathlib
import socket
import subprocess
import sys
import transformers
from typing import *

!pip install markdown_strings
import markdown_strings
from IPython.display import display, Markdown, Latex

import rich
import rich.console
import rich.markdown
import rich.table

import tensorflow.python.framework.ops as ops
import tensorflow as tf
import tensorflow.python.distribute.values as values
import toolz
import tqdm.notebook as tqdm

_PROJECT_DIRECTORY = pathlib.Path().resolve().parent
sys.path.append(str(_PROJECT_DIRECTORY))
import constants
import task_specific
import tf_utils
import utils


In [2]:
def normal(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(text))

def h1(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(f"# {text}"))
    
def h2(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(f"#### {text}"))
    
def quote(text, escape=True):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(markdown_strings.blockquote(text)))
    
def build_split_to_ds_paths(project_directory, num_paths_display):
    h1("Getting filenames.")
    h2("Loading json config.")
    config_path = project_directory/"configs"/"train_configs"/"tpu_gpt2_eli5_kilt.json"
    config = utils.from_json_file(config_path)
    
    h2("Calling `gsutil ls` on the dataset repo.")
    ds_path = config["tfr_prefix"]
    filenames = subprocess.check_output(f"gsutil ls {ds_path}", shell=True).decode().strip().split("\n")

    h2("Printing a few paths:")
    normal(f"There are actually {len(filenames)}.")
    normal(" - " + "\n - ".join(filenames[:num_paths_display]))
    
    h1("Building the `per_split` Path dict.")
    per_split = collections.defaultdict(list)
    for path in tqdm.tqdm(filenames, desc="Building `per_split` dict."):
        split = pathlib.Path(path).name.split("_")[0]
        per_split[split].append(path)

    normal("Sorting the `per_split` lists.")
    for split in per_split:
        # Ad-hoc split per file index
        per_split[split].sort(key=lambda p: int(pathlib.Path(p).name.split("_")[1].split(".")[0]))

    normal("Len per split for the per_split dict:")
    
    print({split: len(per_split[split]) for split in per_split})
    
    return per_split


def build_dataset(paths, context_window_size, split, batch_size):
    ds = tf.data.TFRecordDataset(paths)
    description = {
      constants.CTH5Fields.distances:
          tf.io.FixedLenFeature((), tf.string),
      constants.CTH5Fields.gpt2_retrieved_ids:
          tf.io.FixedLenFeature((), tf.string),
      constants.CTH5Fields.gpt2_question_ids_inputs:
          tf.io.FixedLenFeature((), tf.string),
    }
    if split != constants.SplitChoices.test:
        description[
            constants.CTH5Fields.gpt2_answer_ids_inputs
        ] = tf.io.FixedLenFeature((), tf.string)

    feature_dtypes = {
      constants.CTH5Fields.distances:
          tf.float32,
      constants.CTH5Fields.gpt2_retrieved_ids:
          tf.int32,
      constants.CTH5Fields.gpt2_question_ids_inputs:
          tf.int32,
    }
    if split != constants.SplitChoices.test:
        feature_dtypes[
            constants.CTH5Fields.gpt2_answer_ids_inputs
        ] = tf.int32

    feature_shape = {
      constants.CTH5Fields.distances:
          (10,),
      constants.CTH5Fields.gpt2_retrieved_ids:
          (10, context_window_size,),
      constants.CTH5Fields.gpt2_question_ids_inputs:
          (context_window_size,),
    }
    if split != constants.SplitChoices.test:
        feature_shape[constants.CTH5Fields.gpt2_answer_ids_inputs] = (
            context_window_size
        )

    @tf.function
    def parse(sample):
        example = tf.io.parse_single_example(sample, description)
        output = {}
        for k, v in example.items():
            output[k] = tf.io.parse_tensor(v, out_type=feature_dtypes[k])
            output[k].set_shape(feature_shape[k])
        return output

    ds = ds.map(
      parse,
      num_parallel_calls=tf.data.experimental.AUTOTUNE,
      deterministic=False
      )
    
    ds = ds.batch(
      batch_size,
      drop_remainder=split != constants.SplitChoices.test
      )

    return ds


def decode_line(tokenizer, line):
    return tokenizer.decode([x for x in line if x >= 0])


def is_all_neg(tensor):
    if not isinstance(tensor, (np.ndarray, tf.Tensor, ops.EagerTensor)):
        return all([x < 0 for x in tensor])
    else:
        return np.all(tensor < 0)
    
    
def check_and_decode(feature_key, item, tokenizer):
    feature = item[feature_key]
    all_neg = is_all_neg(feature)
    assert not all_neg, feature_key
    return decode_line(tokenizer, feature)
    
    
def display_item(major, minor, tokenizer, item, split):    
    item = vars(item)
    ##################################################################################################################
    # Produce information
    ##################################################################################################################
    question = check_and_decode(
        constants.CTH5Fields.gpt2_question_ids_inputs,
        item,
        tokenizer
    )
    
    answer = None
    if split != "test":
        feature_key = constants.CTH5Fields.gpt2_answer_ids_inputs
        feature = item[feature_key]
        answer = check_and_decode(feature_key, item, tokenizer)

    retrieved_segments = []
    for line in item[constants.CTH5Fields.gpt2_retrieved_ids]:
        retrieved_segments.append(decode_line(tokenizer, line))

    ##################################################################################################################
    # Display
    ##################################################################################################################
    console = rich.console.Console()
    table = rich.table.Table(title=f"{major}.{minor} - Item from split `{split}`", show_lines=True)
    table.add_column("Field", style="bold")
    table.add_column("Value")
    table.add_row("Question:", question)
    for i, segment in enumerate(retrieved_segments):
        table.add_row(f"Retrieved segment {i}:", segment)

    if answer:
        table.add_row("Answer:", answer)
    
    console.print(table)
    

def check_all_unique(iterable):
    """Memory and computation scale in O(N) with N = len(iterable). """
    
    iter_count = 0
    set_ = set()
    
    for item in iterable:
        iter_count += 1
        set_.add(item) 
    
    utils.check_equal(iter_count, len(set_))

    
@dataclasses.dataclass
class Sample:
    distances: tf.Tensor
    gpt2_answer_ids: tf.Tensor
    gpt2_question_ids: tf.Tensor
    gpt2_retrieved_ids: tf.Tensor

In [3]:
###############################################################################
# Long configuration stuff
###############################################################################

#------------------------------------------------------------------------------
# Flags
#------------------------------------------------------------------------------
_MAX_QTY = None
_MODEL_TYPE = "distilgpt2"
_EXPECTED_SIZES = dict(train=272634, eval=1507, test=600)
_NUM_PATHS_DISPLAY = 10
_NUM_REPLICAS = 8
_ACCEL_TYPE = "TPU"


#------------------------------------------------------------------------------
# TPU Stuff
#------------------------------------------------------------------------------
if _ACCEL_TYPE == "TPU":
    tpu_setup = tf_utils.init_tpus(socket.gethostname())
    utils.check_equal(tf_utils.devices_to_use()[0].device_type, "TPU")
    utils.check_equal(len(tf_utils.devices_to_use()), 8)
    strategy = tf.distribute.TPUStrategy(tpu_setup.resolver)
elif _ACCEL_TYPE == "CPU":
    device = tf_utils.devices_to_use()[0]
    utils.check_equal(len(tf_utils.devices_to_use()), 1)
    utils.check_equal(device.device_type, "CPU")
    strategy = tf.distribute.OneDeviceStrategy(device)
    
else:
    raise RuntimeError(_ACCEL_TYPE)
    
    
#------------------------------------------------------------------------------
# Huggingface Stuff
#------------------------------------------------------------------------------
model_config = transformers.AutoConfig.from_pretrained(_MODEL_TYPE)
tokenizer = transformers.AutoTokenizer.from_pretrained(_MODEL_TYPE)
splits_to_ds_paths = build_split_to_ds_paths(_PROJECT_DIRECTORY, _NUM_PATHS_DISPLAY)
context_window_size = model_config.n_ctx

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: jules


INFO:tensorflow:Initializing the TPU system: jules


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_0.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_10.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_100.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1000.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1001.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1002.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1003.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1004.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1005.tfr

# Building the `per_split` Path dict.

Sorting the `per_split` lists.

Len per split for the per_split dict:

{'eval': 2048, 'test': 2048, 'train': 2048, 'validation': 2048}


In [ ]:
for split in [
# "eval", 
# "test", 
    "train"
]:
    ds_paths = splits_to_ds_paths[split]
    check_all_unique(ds_paths)
    
    ###########################################################################
    # Build and Distribute the DS
    ###########################################################################
    print("Building DS")
    ds = build_dataset(ds_paths, context_window_size, split, len(tf_utils.devices_to_use()))
    ds = ds.batch(
        len(tf_utils.devices_to_use()), drop_remainder=True
    )
    dds = strategy.experimental_distribute_dataset(ds)
    print("Created DS")
    
    ###########################################################################
    # Print elements of the DS
    ###########################################################################
    for major, dist_items in enumerate(toolz.take(2, dds)):
        
        turned_on_itself = [
            Sample(None, None, None, None) for _ in range(len(tf_utils.devices_to_use()))
        ]

        for feature_key, v in dist_items.items():
            if tf_utils.devices_to_use()[0].device_type == "TPU":
                utils.check_isinstance(v, values.PerReplica)
            else:
                utils.check_isinstance(v, ops.EagerTensor)
            
            rich.print(len(v.values))
            rich.print(v.values)
            exit()
            for i, vv in enumerate(v.values if isinstance(v, values.PerReplica) else [v]):
                utils.check_equal(vv.shape[0], 1)
                setattr(turned_on_itself[i], feature_key, vv[0])

                
        utils.check_equal(len(turned_on_itself), len(tf_utils.devices_to_use()))
        for minor, item in enumerate(turned_on_itself):
            rich.print(item)
            if split == "train":
                display_item(major, minor, tokenizer, item, split)

